<a href="https://colab.research.google.com/github/j00lee/SignLingo/blob/main/MS_Citizen_Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up Google Drive Access

In [ ]:
# Set up Google Drive access
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Processing the Videos

In [ ]:
import cv2
import os
from multiprocessing import Pool
from tqdm import tqdm

In [ ]:
# These are the paths
videos_path = '/content/drive/MyDrive/videos'
frames_output_path = '/content/drive/MyDrive/frames'

# Create the frames output folder if it doesn't exist
os.makedirs(frames_output_path, exist_ok=True)

# How often to take a frame
frame_skip = 20

# List all the video files
video_files = [f for f in os.listdir(videos_path) if f.endswith('.mp4')]

# Process each video
def process_video(video_file):
    try:
        video_path = os.path.join(videos_path, video_file)

        # Create a folder for this video's frames
        video_name = os.path.splitext(video_file)[0]
        video_frames_folder = os.path.join(frames_output_path, video_name)
        os.makedirs(video_frames_folder, exist_ok=True)

        cap = cv2.VideoCapture(video_path)
        frame_idx = 0
        saved_idx = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            if frame_idx % frame_skip == 0:
                # Optional resize
                # frame = cv2.resize(frame, resize_shape)

                frame_filename = os.path.join(video_frames_folder, f'frame_{saved_idx:04d}.jpg')
                cv2.imwrite(frame_filename, frame)
                saved_idx += 1

            frame_idx += 1

        cap.release()

    except Exception as e:
        print(f"Failed processing {video_file}: {e}")

# Number of processes (workers)
num_workers = 4

# Start multiprocessing pool
with Pool(num_workers) as p:
    list(tqdm(p.imap_unordered(process_video, video_files), total=len(video_files)))


100%|██████████| 83399/83399 [6:42:55<00:00,  3.45it/s]


# Making the train, val, test splits

In [ ]:
import os
import shutil
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool

# Paths
frames_root = '/content/drive/MyDrive/frames'
dataset_root = '/content/drive/MyDrive/dataset'
os.makedirs(dataset_root, exist_ok=True)

# Create subfolders
splits = ['train', 'val', 'test']
for split in splits:
    os.makedirs(os.path.join(dataset_root, split), exist_ok=True)

# Load CSVs
train_videos = pd.read_csv('/content/drive/MyDrive/splits/train.csv')
val_videos = pd.read_csv('/content/drive/MyDrive/splits/val.csv')
test_videos = pd.read_csv('/content/drive/MyDrive/splits/test.csv')

# Get clean video lists (remove .mp4)
train_list = [os.path.splitext(x)[0] for x in train_videos['Video file'].tolist()]
val_list = [os.path.splitext(x)[0] for x in val_videos['Video file'].tolist()]
test_list = [os.path.splitext(x)[0] for x in test_videos['Video file'].tolist()]

# Prepare (video_name, split_name) pairs
all_videos = [(v, 'train') for v in train_list] + \
             [(v, 'val') for v in val_list] + \
             [(v, 'test') for v in test_list]

# Function to move/copy a single video folder
def move_single_video(args):
    video_name, split_name = args
    src = os.path.join(frames_root, video_name)
    dst = os.path.join(dataset_root, split_name, video_name)

    if os.path.exists(src):
        try:
            shutil.copytree(src, dst)  # or shutil.move(src, dst) if you want to move
        except Exception as e:
            print(f"❗ Error moving {video_name}: {e}")
    else:
        print(f"⚠️ Warning: {src} does not exist!")

# Parallel moving using multiprocessing
num_workers = 1  # You can tune this (4, 6, 8 depending on colab/mac resources)

with Pool(num_workers) as p:
    list(tqdm(p.imap_unordered(move_single_video, all_videos), total=len(all_videos)))

print("✅ Done splitting the dataset with multiprocessing!")


In [ ]:
import os
import shutil
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool

# === STEP 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# === STEP 2: Copy frames from Drive to Colab local disk (/content/frames_local)
print("📂 Copying frames from Drive to local disk...")
!mkdir -p /content/frames_local

source_root = '/content/drive/MyDrive/frames'
dest_root = '/content/frames_local'

# Create destination folder if not exists
os.makedirs(dest_root, exist_ok=True)

# List all folders in frames/
frame_folders = os.listdir(source_root)

# Copy each folder one by one (with progress bar)
for folder in tqdm(frame_folders, desc="Copying folders to local disk"):
    src_folder = os.path.join(source_root, folder)
    dst_folder = os.path.join(dest_root, folder)

    if os.path.isdir(src_folder):
        try:
            shutil.copytree(src_folder, dst_folder)
        except Exception as e:
            print(f"Error copying {folder}: {e}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📂 Copying frames from Drive to local disk...


Copying folders to local disk:  29%|██▉       | 23989/83399 [01:44<00:46, 1268.48it/s]

Error copying 8035548352616286-SODA: [('/content/drive/MyDrive/frames/8035548352616286-SODA/frame_0003.jpg', '/content/frames_local/8035548352616286-SODA/frame_0003.jpg', '[Errno 107] Transport endpoint is not connected'), ('/content/drive/MyDrive/frames/8035548352616286-SODA/frame_0004.jpg', '/content/frames_local/8035548352616286-SODA/frame_0004.jpg', "[Errno 107] Transport endpoint is not connected: '/content/drive/MyDrive/frames/8035548352616286-SODA/frame_0004.jpg'"), ('/content/drive/MyDrive/frames/8035548352616286-SODA', '/content/frames_local/8035548352616286-SODA', "[Errno 107] Transport endpoint is not connected: '/content/drive/MyDrive/frames/8035548352616286-SODA'")]


Copying folders to local disk: 100%|██████████| 83399/83399 [01:45<00:00, 792.01it/s] 


In [ ]:
drive.flush_and_unmount()
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# === STEP 3: Setup local working paths
frames_root = '/content/frames_local'
dataset_root = '/content/dataset'  # Local directory for processing

# Create train/val/test folders
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(dataset_root, split), exist_ok=True)

# === STEP 4: Load CSVs
train_videos = pd.read_csv('/content/drive/MyDrive/splits/train.csv')
val_videos = pd.read_csv('/content/drive/MyDrive/splits/val.csv')
test_videos = pd.read_csv('/content/drive/MyDrive/splits/test.csv')

# Get video lists (remove ".mp4" extension)
train_list = [os.path.splitext(x)[0] for x in train_videos['Video file'].tolist()]
val_list = [os.path.splitext(x)[0] for x in val_videos['Video file'].tolist()]
test_list = [os.path.splitext(x)[0] for x in test_videos['Video file'].tolist()]

# All videos to process
all_videos = [(v, 'train') for v in train_list] + \
             [(v, 'val') for v in val_list] + \
             [(v, 'test') for v in test_list]

# === STEP 5: Move each video folder locally
# Add missing counter
from multiprocessing import Value, Lock

missing_counter = Value('i', 0)  # Shared counter across processes
counter_lock = Lock()

def move_single_video(args):
    video_name, split_name = args
    src = os.path.join(frames_root, video_name)
    dst = os.path.join(dataset_root, split_name, video_name)

    if os.path.exists(src):
        try:
            shutil.copytree(src, dst)
        except Exception as e:
            print(f"❗ Error moving {video_name}: {e}")
    else:
        print(f"⚠️ Warning: {src} does not exist!")
        with counter_lock:
            missing_counter.value += 1

# Multiprocessing copy with few workers (safe for local disk)
print("🚀 Splitting dataset into train/val/test on local disk...")
num_workers = 2

with Pool(num_workers) as p:
    list(tqdm(p.imap_unordered(move_single_video, all_videos), total=len(all_videos)))

# === NEW: After moving, report how many missing
print(f"⚡ Total missing videos: {missing_counter.value}")

print("✅ Done splitting dataset locally!")

# === STEP 6: Zip each split folder
print("📦 Zipping train/val/test folders...")
for split in ['train', 'val', 'test']:
    split_path = os.path.join(dataset_root, split)
    zip_path = f'/content/{split}.zip'
    shutil.make_archive(base_name=zip_path.replace('.zip', ''), format='zip', root_dir=split_path)
    print(f"✅ Zipped {split}")

# === STEP 7: Create destination folder on Drive if needed
!mkdir -p /content/drive/MyDrive/dataset_zips

# === STEP 8: Move zipped files back to Google Drive
print("⬆️ Uploading zipped splits back to Google Drive...")
!mv /content/train.zip /content/drive/MyDrive/dataset_zips/train.zip
!mv /content/val.zip /content/drive/MyDrive/dataset_zips/val.zip
!mv /content/test.zip /content/drive/MyDrive/dataset_zips/test.zip

print("🎉 All done! Train/Val/Test splits are zipped and saved in Drive!")

🚀 Splitting dataset into train/val/test on local disk...


  0%|          | 0/83399 [00:00<?, ?it/s]

⚠️ Warning: /content/frames_local/15890366051589533-APPLE does not exist!⚠️ Warning: /content/frames_local/35618482303951104-IMPOSSIBLE does not exist!

⚠️ Warning: /content/frames_local/8006032738002744-SOCCER 2 does not exist!⚠️ Warning: /content/frames_local/6958143575951994-PARK does not exist!

⚠️ Warning: /content/frames_local/37542279833186454-STINK does not exist!⚠️ Warning: /content/frames_local/9498964909837952-ROSE does not exist!
⚠️ Warning: /content/frames_local/9618808117704574-HUG does not exist!

⚠️ Warning: /content/frames_local/15543213389651633-TUBE does not exist!⚠️ Warning: /content/frames_local/9162908944303221-COUNT does not exist!

⚠️ Warning: /content/frames_local/14154575310664197-NICE does not exist!
⚠️ Warning: /content/frames_local/996512226187074-EASY TO_DO does not exist!⚠️ Warning: /content/frames_local/0033742275375785447-INFLUENCE does not exist!



  0%|          | 11/83399 [00:00<12:39, 109.76it/s]

⚠️ Warning: /content/frames_local/02808154918358552-PEACOCK does not exist!⚠️ Warning: /content/frames_local/8068093518563271-CIRCUSWHEEL does not exist!

⚠️ Warning: /content/frames_local/23247991626849585-GYM does not exist!
⚠️ Warning: /content/frames_local/6652814499755777-NOSE does not exist!⚠️ Warning: /content/frames_local/20880000068819649-WEAVE does not exist!
⚠️ Warning: /content/frames_local/13621431540213602-SMOOTH does not exist!

⚠️ Warning: /content/frames_local/4520498201410337-BACKPACK does not exist!⚠️ Warning: /content/frames_local/16994877911771145-WE does not exist!

⚠️ Warning: /content/frames_local/7090729284366466-DOWNHILL does not exist!⚠️ Warning: /content/frames_local/19003684436528423-JUICE 2 does not exist!
⚠️ Warning: /content/frames_local/8221969382671432-SCOLD does not exist!
⚠️ Warning: /content/frames_local/12338257144617781-POWDER does not exist!
⚠️ Warning: /content/frames_local/7480838717347633-SOCIAL does not exist!

⚠️ Warning: /content/frames_loc

...


⚠️ Warning: /content/frames_local/1567605325168595-FAR does not exist!⚠️ Warning: /content/frames_local/633464020430156-HORRIBLE does not exist!

⚠️ Warning: /content/frames_local/08758415349864745-OUT does not exist!
⚠️ Warning: /content/frames_local/7896700330944251-DEMAND does not exist!⚠️ Warning: /content/frames_local/29173298416034354-PRESS IN_MIND does not exist!
⚠️ Warning: /content/frames_local/47363744282826903-CONTACT does not exist!


  9%|▉         | 7742/83399 [00:49<08:05, 155.89it/s]

⚠️ Warning: /content/frames_local/7487799802593937-HURT does not exist!

Process ForkPoolWorker-2:
Process ForkPoolWorker-1:



KeyboardInterrupt: 

In [ ]:
drive.flush_and_unmount()
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool

# === Paths
frames_root = '/content/drive/MyDrive/frames'  # <-- work directly from Drive
dataset_root = '/content/drive/MyDrive/dataset'  # <-- split inside Drive

# Create train/val/test folders
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(dataset_root, split), exist_ok=True)

# === Load CSVs
train_videos = pd.read_csv('/content/drive/MyDrive/splits/train.csv')
val_videos = pd.read_csv('/content/drive/MyDrive/splits/val.csv')
test_videos = pd.read_csv('/content/drive/MyDrive/splits/test.csv')

# === Get video lists (remove ".mp4" extension)
train_list = [os.path.splitext(x)[0] for x in train_videos['Video file'].tolist()]
val_list = [os.path.splitext(x)[0] for x in val_videos['Video file'].tolist()]
test_list = [os.path.splitext(x)[0] for x in test_videos['Video file'].tolist()]

# Combine all needed moves
all_videos = [(v, 'train') for v in train_list] + \
             [(v, 'val') for v in val_list] + \
             [(v, 'test') for v in test_list]

# === Move each video folder directly on Drive
# Add missing counter
from multiprocessing import Value, Lock

missing_counter = Value('i', 0)
counter_lock = Lock()

def move_single_video(args):
    video_name, split_name = args
    src = os.path.join(frames_root, video_name)
    dst = os.path.join(dataset_root, split_name, video_name)

    if os.path.exists(src):
        try:
            shutil.copytree(src, dst)
        except Exception as e:
            print(f"❗ Error moving {video_name}: {e}")
    else:
        print(f"⚠️ Warning: {src} does not exist!")
        with counter_lock:
            missing_counter.value += 1

# Multiprocessing with careful number of workers (small to avoid Drive throttling)
print("🚀 Splitting dataset into train/val/test directly on Drive...")
num_workers = 2  # Safe for Drive access

with Pool(num_workers) as p:
    list(tqdm(p.imap_unordered(move_single_video, all_videos), total=len(all_videos)))

print(f"⚡ Total missing videos: {missing_counter.value}")
print("✅ Done splitting dataset directly on Drive!")


🚀 Splitting dataset into train/val/test directly on Drive...


 91%|█████████ | 76088/83399 [11:53:27<1:51:07,  1.10it/s]